In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.0.1+cu117
Torchvision version: 0.15.2+cu117
CUDA is available: True


In [3]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:100000"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
os.getcwd()

'/home/yelhe/script/mt'

In [4]:
import glob
import shapely
from shapely.geometry import Polygon
import shapely.plotting

In [5]:
import pandas as pd
from PIL import Image as ima
import warnings
from pathlib import Path
import random

In [6]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

# Add function def show_anns (borrowed from one of the Jupyter notebooks).
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    # polygons = []
    # color = []
    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)
    # for ann in sorted_anns:
    #     m = ann['segmentation']
    #     img = np.ones((m.shape[0], m.shape[1], 3))
    #     color_mask = np.random.random((1, 3)).tolist()[0]
    #     for i in range(3):
    #         img[:,:,i] = color_mask[i]
    #     ax.imshow(np.dstack((img, m*0.35)))

In [90]:
# from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
# sam = sam_model_registry["vit_h"](checkpoint="/home/yelhe/data/sam/sam_vit_h_4b8939.pth")
# mask_generator = SamAutomaticMaskGenerator(sam)


### Data preparation

In [7]:
koord  = pd.read_csv('/home/yelhe/script/mt/output/koord.csv')

In [8]:
# koord.head()
name1 = koord.Nr.astype(str) + '1.jpg'
name2 = koord.Nr.astype(str) + '2.jpg'
name3 = koord.Nr.astype(str) + '3.jpg'
name4 = koord.Nr.astype(str) + '4.jpg'

In [9]:
gsv_image1 = []
gsv_image2 = []
gsv_image3 = []
gsv_image4 = []

nf1 = []
nf2 = []
nf3 = []
nf4 = []
for i in range(0,349):
    path1 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name1[i])
    c1 = path1.is_file()
    path2 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name2[i])
    c2 = path2.is_file()
    path3 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name3[i])
    c3 = path3.is_file()
    path4 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name4[i])
    c4 = path4.is_file()
    if c1 is True:
        im1 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name1[i])
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
        gsv_image1.append(im1)
        nf1.append(name1[i])
    if c2 is True:
        im2 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name2[i])
        im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
        gsv_image2.append(im2)
        nf2.append(name2[i])
    if c3 is True:
        im3 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name3[i])
        im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)
        gsv_image3.append(im3)
        nf3.append(name3[i])
    if c4 is True:
        im4 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name4[i])
        im4 = cv2.cvtColor(im4, cv2.COLOR_BGR2RGB)
        gsv_image4.append(im4)
        nf4.append(name4[i])

In [10]:
print(len(gsv_image1),len(gsv_image2),len(gsv_image3),len(gsv_image4))

295 295 295 295


In [11]:
n1 = [s.replace('.jpg', '') for s in nf1]
n2 = [s.replace('.jpg', '') for s in nf2]
n3 = [s.replace('.jpg', '') for s in nf3]
n4 = [s.replace('.jpg', '') for s in nf4]

### Functions preparation

In [12]:
def convert_mask(masks):
    mb01 = []
    mb02 = []
    mb = []
    n = len(masks)
    for i in range(0, n):
        mb01.append(masks[i]['segmentation'])
    for i in range(0, n):
        mb02.append(mb01[i]*1)
    for i in range(0, n):
        mb.append(mb02[i].astype('uint8'))
    return mb

In [13]:
def output_mask(im):
    im1 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    mask_r = mask_generator.generate(im1)
    mask_c = convert_mask(mask_r)
    return mask_c

In [14]:
# Updated Function
# Function to find and plot filtered contours (find-filter-plot-contour)
# Run for each mask of each image
def ffpcontour_noplot(image, mask, i):
    assert image is not None, "image file could not be read, check with os.path.exists()"
    assert mask is not None, "mask file could not be read, check with os.path.exists()"
    # imgray = cv2.cvtColor(image_masked, cv2.COLOR_BGR2GRAY)
    # ret, thresh = cv2.threshold(imgray, 127, 255, 0)
    ret, thresh = cv2.threshold((mask[i]*255), 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ll = [] # length list
    for i in range(0, len(contours)):
          ll.append(len(contours[i]))
    maxl = max(ll)
    maxindex = ll.index(maxl)
    if (len(contours) == 1) and (maxl >= 6):
        contour_f = contours
    elif (maxl>=80) :
        contour_f = []
        for i in range(0, len(contours)):
        # print(i, "len", len(contours[i]))
            if (len(contours[i]) >= 80):
                contour_f.append(contours[i])
            else:
                contour_f = contour_f
    elif (maxl >= 6):
        contour_f = []
        contour_f.append(contours[maxindex])
    else: 
        contour_f = []
    # print("filtered", "len", len(contour_f), contour_f) 
    # print("raw",len(contours))
    # print("filtered", len(contour_f))
    return contour_f

In [15]:
# Contour geometric attributes result
# For each contour
def cgr(contour):
    assert contour is not None, "image file could not be read, check with os.path.exists()"
    c = contour
    isconvex = cv2.isContourConvex(c) # Checking convexity
    (x,y), (w,h), ar = cv2.minAreaRect(c) # Rotated rectangle with minimum area
    M = cv2.moments(c) # Moments
    area = cv2.contourArea(c) # Area 
    if (M['m00'] != 0):
        cx = int(M['m10']/M['m00']) # Centroid
        cy = int(M['m01']/M['m00'])
    else:
        cx = x
        cy = y
    xs,ys,ws,hs = cv2.boundingRect(c) # Straight bounding rectangle
    aspect_ratio_wh_s = float(ws)/hs # Aspect ratio
    extent_s = float(area)/(ws*hs) # Extent
    hull = cv2.convexHull(c) # Solidity
    hull_area = cv2.contourArea(hull)
    if (hull_area != 0):
        solidity = float(area)/hull_area
    else:
        solidity = 0
    aspect_ratio_wh = float(w)/h  # Aspect ratio
    extent = float(area)/(w*h) # Extent
    (xe,ye),(MA,ma),ae = cv2.fitEllipse(c)
    ed = np.sqrt(4*area/np.pi) # Equivalent Diameter
    ratio_ell = float(ma)/MA
    perimeter = cv2.arcLength(c, True) # Arclength
    p_centroid = np.array([float(cx), float(cy)])
    p_masscenter = np.array([float(x), float(y)])
    is_cen_inside = cv2.pointPolygonTest(c, p_centroid, False) # Checking if centroid is inside
    is_mce_inside = cv2.pointPolygonTest(c, p_masscenter, False) # Checking if mass center is inside    
    return {
        'isconvex': isconvex,
        'area': area,
        'aspect_ratio_wh_s': aspect_ratio_wh_s,
        'extent_s': extent_s,
        'solidity': solidity,
        'aspect_ratio_wh': aspect_ratio_wh,
        'extent': extent,
        'ed': ed,
        'ratio_ell': ratio_ell,
        'perimeter': perimeter,
        'is_cen_inside': is_cen_inside,
        'is_mce_inside': is_mce_inside
    }
          

In [16]:
def csga(contours):
    assert contours is not None, "image file could not be read, check with os.path.exists()"
    if len(contours) == 1:
        ga = cgr(contours[0])
    else:
        gal = []
        for i in range(0, (len(contours)-1)):
            gal.append(cgr(contours[i]))
        isc = []
        al = []
        asps = []
        exts = []
        sol = []
        asp = []
        ext = []
        ed = []
        rate = []
        per = []
        isc = []
        ism = []
        for i in range(0, len(gal)):
            isc.append(gal[0]['isconvex'])
            al.append(gal[0]['area'])
            asps.append(gal[0]['aspect_ratio_wh_s'])
            exts.append(gal[0]['extent_s'])
            sol.append(gal[0]['solidity'])
            asp.append(gal[0]['aspect_ratio_wh'])
            ext.append(gal[0]['extent'])
            ed.append(gal[0]['ed'])
            rate.append(gal[0]['ratio_ell'])
            per.append(gal[0]['perimeter'])
            isc.append(gal[0]['is_cen_inside'])
            ism.append(gal[0]['is_mce_inside'])
        isconvex = np.all(isc)
        area = np.mean(al, axis = 0)
        aspect_ratio_wh_s = np.mean(asps, axis = 0)
        extent_s = np.mean(exts, axis = 0)
        solidity = np.mean(sol, axis = 0)
        aspect_ratio_wh = np.mean(asp, axis = 0)
        extent = np.mean(ext, axis = 0)
        ed = np.mean(ed, axis = 0)
        ratio_ell = np.mean(rate, axis = 0)
        perimeter = np.mean(per, axis = 0)
        is_cen_inside = np.mean(isc, axis = 0)
        is_mce_inside = np.mean(ism, axis = 0)
        ga = {
            'isconvex': isconvex,
            'area': area,
            'aspect_ratio_wh_s': aspect_ratio_wh_s,
            'extent_s': extent_s,
            'solidity': solidity,
            'aspect_ratio_wh': aspect_ratio_wh,
            'extent': extent,
            'ed': ed,
            'ratio_ell': ratio_ell,
            'perimeter': perimeter,
            'is_cen_inside': is_cen_inside,
            'is_mce_inside': is_mce_inside
        }
    return ga


In [17]:
# Updated - Remove the empty contour
# mask file mf
def feature_summary(image, mf):
    # Generate a data frame for masks and attributes
    df = pd.DataFrame()
    df['mask'] = range(1, (len(mf)+1))
    df = df.assign(gmedian = None, rmedian = None, bmedian = None,
                   gmean = None, rmean = None, bmean = None,
                   gstd = None, rstd = None, bstd = None,
                   isconvex = None, area = None, aspect_ratio_wh_s = None,
                   extent_s = None, solidity = None, aspect_ratio_wh = None,
                   extent = None, ed = None, ratio_ell = None,
                   perimeter = None, is_cen_inside = None, is_mce_inside = None)
    mm = [] # masked image
    for i in range(0, len(mf)):
        mm.append(cv2.bitwise_and(image, image, mask = mf[i]))
    mib = []
    mig = []
    mir = []
    for i in range(0, len(mm)):
        mib.append((mm[i][:,:,0])[np.where((mm[i][:,:,0]) != 0)])
        mig.append((mm[i][:,:,1])[np.where((mm[i][:,:,1]) != 0)])
        mir.append((mm[i][:,:,2])[np.where((mm[i][:,:,2]) != 0)])
    # with warnings.catch_warnings():
    #     warnings.simplefilter("ignore", category=RuntimeWarning)
    #     foo = np.nanmean(x, axis=1)
    for i in range(0, len(mm)):
        df.at[i, 'bmean'] = np.mean(mib[i], axis = 0)
        df.at[i,'gmean'] = np.mean(mig[i], axis = 0)
        df.at[i,'rmean'] = np.mean(mir[i], axis = 0)
        df.at[i,'bmedian'] = np.median(mib[i], axis = 0)
        df.at[i,'gmedian'] = np.median(mig[i], axis = 0)
        df.at[i,'rmedian'] = np.median(mir[i], axis = 0)
        df.at[i,'bstd'] = np.std(mib[i], axis = 0)
        df.at[i,'gstd'] = np.std(mig[i], axis = 0)
        df.at[i,'rstd'] = np.std(mir[i], axis = 0)
    for i in range(0, len(mf)):
        con = ffpcontour_noplot(image, mf, i)
        if len(con) != 0 :
            df.at[i, 'isconvex'] = csga(con)['isconvex']
            df.at[i, 'area'] = csga(con)['area']
            df.at[i, 'aspect_ratio_wh_s'] = csga(con)['aspect_ratio_wh_s']
            df.at[i, 'extent_s'] = csga(con)['extent_s']
            df.at[i, 'solidity'] = csga(con)['solidity']
            df.at[i, 'aspect_ratio_wh'] = csga(con)['aspect_ratio_wh']
            df.at[i, 'extent'] = csga(con)['extent']
            df.at[i, 'ed'] = csga(con)['ed']
            df.at[i, 'ratio_ell'] = csga(con)['ratio_ell']
            df.at[i, 'perimeter'] = csga(con)['perimeter']
            df.at[i, 'is_cen_inside'] = csga(con)['is_cen_inside']
            df.at[i, 'is_mce_inside'] = csga(con)['is_mce_inside']
        else :
            df.at[i, 'isconvex'] = np.nan
            df.at[i, 'area'] = np.nan
            df.at[i, 'aspect_ratio_wh_s'] = np.nan
            df.at[i, 'extent_s'] = np.nan
            df.at[i, 'solidity'] = np.nan
            df.at[i, 'aspect_ratio_wh'] = np.nan
            df.at[i, 'extent'] = np.nan
            df.at[i, 'ed'] = np.nan
            df.at[i, 'ratio_ell'] = np.nan
            df.at[i, 'perimeter'] = np.nan
            df.at[i, 'is_cen_inside'] = np.nan
            df.at[i, 'is_mce_inside'] = np.nan
    # Remove the rows with na
    df = df.dropna()
    return df

In [18]:
def filter_overlap(mask):
    mb_new = mask
    mb_new1 = mask
    for i in range(0, (len(mb_new)-1)):
        for j in range((i+1), len(mb_new)):
            a = cv2.bitwise_and(mb_new[i], mb_new[j])
            al = len(np.unique(a))
            # print(i, j, "len", al)
            if al != 1:
                s1 = cv2.countNonZero(mb_new[i])
                s2 = cv2.countNonZero(mb_new[j])
                b = cv2.bitwise_or(mb_new[i], mb_new[j])
                # print(i,j, "have intersection")
                s3 = cv2.countNonZero(b)
                if s1 >= s2 :
                    # print(i,">", j)
                    if s1 == s3:
                        # print(j, "subset of", i)
                        mb_new1[j] = 0
                else:
                    # print(i,"<", j)
                    if s2 == s3:
                        # print(i, "subset of", j)
                        mb_new1[i] = 0
    emptyl = []
    for i in range(0, len(mb_new)):
        if np.all(mb_new1[i] == 0):
            emptyl.append(i)           
    if len(emptyl) != 0 :
        mb_new1 = np.delete(mb_new1, emptyl, 0)
    else:
        mb_new1 = mb_new1
    # print(len(mask), len(mb_new1))
    return mb_new1


### Run workflow on generated gsv mask files

In [991]:
mask1 = []
mask2 = []
mask3 = []
mask4 = []
# Load all generated mask files

for i in range(0,295):
    m1 = np.load('/home/yelhe/data/gsv_output/' + n1[i] + '.npy', allow_pickle=True)
    mask1.append(m1)
    m2 = np.load('/home/yelhe/data/gsv_output/' + n2[i] + '.npy', allow_pickle=True)
    mask2.append(m2)
    m3 = np.load('/home/yelhe/data/gsv_output/' + n3[i] + '.npy', allow_pickle=True)
    mask3.append(m3)
    m4 = np.load('/home/yelhe/data/gsv_output/' + n4[i] + '.npy', allow_pickle=True)
    mask4.append(m4)


print(len(mask1), len(mask2), len(mask3), len(mask4))

295 295 295 295


In [992]:
m1f = []
m2f = []
m3f = []
m4f = []

# Load all generated filtered mask files
for i in range(0,295):
    m1 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n1[i] + 'f.npy', allow_pickle=True)
    m1f.append(m1)
    m2 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n2[i] + 'f.npy', allow_pickle=True)
    m2f.append(m2)
    m3 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n3[i] + 'f.npy', allow_pickle=True)
    m3f.append(m3)
    m4 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n4[i] + 'f.npy', allow_pickle=True)
    m4f.append(m4)

print(len(m1f), len(m2f), len(m3f), len(m4f))

295 295 295 295


In [57]:
# ra = list(range(0, 295))
# # Random choose accidents
# il = random.sample(ra, 10)
# print(il)

[283, 193, 38, 224, 27, 66, 174, 77, 160, 182]


In [19]:
il = [283, 193, 38, 224, 27, 66, 174, 77, 160, 182]

In [20]:
print(il)

[283, 193, 38, 224, 27, 66, 174, 77, 160, 182]


In [21]:
nlist = pd.DataFrame()

In [22]:
nlist['n1'] = n1

In [23]:
nlist['n'] = nlist.n1.str[-15:-1]

In [24]:
nlist = nlist.drop(['n1'], axis=1)

In [25]:
nlist_s = nlist.iloc[il]

In [26]:
nlist_s

,n
283,99900046541031
193,99900058252952
38,99900050934899
224,99900052420710
27,99900046484531
66,99900056647155
174,99900048160516
77,99900055778422
160,99900051076121
182,99900062245276


In [27]:
r1 = nlist_s.n.astype(str) + '1'
r2 = nlist_s.n.astype(str) + '2'
r3 = nlist_s.n.astype(str) + '3'
r4 = nlist_s.n.astype(str) + '4'

In [28]:
r1 = [str(element) for element in r1]
r2 = [str(element) for element in r2]
r3 = [str(element) for element in r3]
r4 = [str(element) for element in r4]

In [29]:
r1

['999000465410311',
 '999000582529521',
 '999000509348991',
 '999000524207101',
 '999000464845311',
 '999000566471551',
 '999000481605161',
 '999000557784221',
 '999000510761211',
 '999000622452761']

In [31]:
mr1 = []
mr2 = []
mr3 = []
mr4 = []
for i in range(0,10):
    m1 = np.load('/home/yelhe/script/mt/random/' + r1[i] + '.npy', allow_pickle=True)
    mr1.append(m1)
    m2 = np.load('/home/yelhe/script/mt/random/' + r2[i] + '.npy', allow_pickle=True)
    mr2.append(m2)
    m3 = np.load('/home/yelhe/script/mt/random/' + r3[i] + '.npy', allow_pickle=True)
    mr3.append(m3)
    m4 = np.load('/home/yelhe/script/mt/random/' + r4[i] + '.npy', allow_pickle=True)
    mr4.append(m4)

print(len(mr1), len(mr2), len(mr3), len(mr4))

10 10 10 10


In [30]:
mr1f = []
mr2f = []
mr3f = []
mr4f = []
for i in range(0,10):
    m1 = np.load('/home/yelhe/script/mt/random/' + r1[i] + 'f.npy', allow_pickle=True)
    mr1f.append(m1)
    m2 = np.load('/home/yelhe/script/mt/random/' + r2[i] + 'f.npy', allow_pickle=True)
    mr2f.append(m2)
    m3 = np.load('/home/yelhe/script/mt/random/' + r3[i] + 'f.npy', allow_pickle=True)
    mr3f.append(m3)
    m4 = np.load('/home/yelhe/script/mt/random/' + r4[i] + 'f.npy', allow_pickle=True)
    mr4f.append(m4)

print(len(mr1f), len(mr2f), len(mr3f), len(mr4f))

10 10 10 10


In [57]:
for i in range(0,10):
    print(i, '1', len(mr1[i]), len(mr1f[i]))
    print(i, '2', len(mr2[i]), len(mr2f[i]))
    print(i, '3', len(mr3[i]), len(mr3f[i]))
    print(i, '4', len(mr4[i]), len(mr4f[i]))

0 1 86 67
0 2 77 46
0 3 68 47
0 4 55 41
1 1 149 76
1 2 117 51
1 3 138 76
1 4 167 90
2 1 92 55
2 2 113 73
2 3 113 80
2 4 77 37
3 1 89 58
3 2 121 44
3 3 93 40
3 4 56 47
4 1 59 44
4 2 57 45
4 3 58 44
4 4 72 52
5 1 135 20
5 2 108 43
5 3 88 46
5 4 93 51
6 1 93 52
6 2 124 88
6 3 102 63
6 4 107 40
7 1 74 53
7 2 49 41
7 3 56 46
7 4 72 57
8 1 103 52
8 2 57 29
8 3 59 45
8 4 80 39
9 1 140 56
9 2 116 62
9 3 112 37
9 4 120 43


In [993]:
for i in range(0,10):
    a = il[i]
    print(i, '1', len(mask1[a]), len(m1f[a]))
    print(i, '2', len(mask2[a]), len(m2f[a]))
    print(i, '3', len(mask3[a]), len(m3f[a]))
    print(i, '4', len(mask4[a]), len(m4f[a]))

0 1 85 54
0 2 81 57
0 3 65 49
0 4 53 39
1 1 155 84
1 2 123 53
1 3 127 79
1 4 177 106
2 1 98 54
2 2 118 75
2 3 117 79
2 4 78 40
3 1 90 59
3 2 122 38
3 3 87 39
3 4 55 51
4 1 54 46
4 2 55 43
4 3 56 40
4 4 72 54
5 1 134 32
5 2 104 50
5 3 91 43
5 4 99 54
6 1 93 55
6 2 130 79
6 3 104 60
6 4 112 39
7 1 84 61
7 2 48 41
7 3 53 44
7 4 76 59
8 1 109 50
8 2 61 31
8 3 60 45
8 4 79 37
9 1 142 54
9 2 121 61
9 3 114 36
9 4 120 40


In [34]:
df1r = []
df2r = []
df3r = []
df4r = []
for i in range(0, 10):
    a = il[i]
    print(i, a)
    df1r.append(feature_summary(gsv_image1[a], m1f[a]))
    df2r.append(feature_summary(gsv_image2[a], m2f[a]))
    df3r.append(feature_summary(gsv_image3[a], m3f[a]))
    df4r.append(feature_summary(gsv_image4[a], m4f[a]))


0 283
1 193
2 38
3 224
4 27
5 66
6 174
7 77
8 160
9 182


In [2146]:
j = j+1
q = il[j]
print(j, q, r3[j])
mb = m3f[q]
img = gsv_image3[q]
df3r[j].info()

9 182 999000622452762
<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 0 to 60
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   mask               60 non-null     int64 
 1   gmedian            60 non-null     object
 2   rmedian            60 non-null     object
 3   bmedian            60 non-null     object
 4   gmean              60 non-null     object
 5   rmean              60 non-null     object
 6   bmean              60 non-null     object
 7   gstd               60 non-null     object
 8   rstd               60 non-null     object
 9   bstd               60 non-null     object
 10  isconvex           60 non-null     object
 11  area               60 non-null     object
 12  aspect_ratio_wh_s  60 non-null     object
 13  extent_s           60 non-null     object
 14  solidity           60 non-null     object
 15  aspect_ratio_wh    60 non-null     object
 16  extent             60 non-nul

In [2207]:
t = 0
p = ((df3r[j]).iloc[t]['mask']) -1
# print(t, p)
i = p
#i = 0
print(i, t,'label')
maskedimg = cv2.bitwise_and(img,img,mask = mb[i])
# create a mask
hist_full = cv2.calcHist([img],[0],None,[256],[0,256])
hist_mask = cv2.calcHist([img],[0],mb[i],[256],[0,256])
plt.figure(figsize = (10,10))
plt.subplot(221), plt.imshow(img)
plt.subplot(222), plt.imshow(mb[i])
plt.subplot(223), plt.imshow(maskedimg)
plt.subplot(224), plt.plot(hist_full), plt.plot(hist_mask)
plt.xlim([0,256])
plt.show()

IndexError: single positional indexer is out-of-bounds

In [38]:
# i = i+1
# print(i)
# plt.figure(figsize = (10,10))
# plt.subplot(421), plt.imshow(gsv_image1[i])
# plt.subplot(422), plt.imshow(m1c[i][0])
# plt.subplot(423), plt.imshow(gsv_image2[i])
# plt.subplot(424), plt.imshow(m2c[i][0])
# plt.subplot(425), plt.imshow(gsv_image3[i])
# plt.subplot(426), plt.imshow(m3c[i][0])
# plt.subplot(427), plt.imshow(gsv_image4[i])
# plt.subplot(428), plt.imshow(m4c[i][0])
# plt.show()

In [39]:
# for i in range(0,10):
#     m = il[i]
#     m1 = np.load('/home/yelhe/script/mt/random/' + n1[m] + '.npy', allow_pickle=True)
#     m1f = filter_overlap(m1)
#     np.save('/home/yelhe/script/mt/random/' + n1[m] + 'f.npy', m1f, allow_pickle=True)
#     m2 = np.load('/home/yelhe/script/mt/random/' + n2[m] + '.npy', allow_pickle=True)
#     m2f = filter_overlap(m2)
#     np.save('/home/yelhe/script/mt/random/' + n2[m] + 'f.npy', m2f, allow_pickle=True)
#     m3 = np.load('/home/yelhe/script/mt/random/' + n3[m] + '.npy', allow_pickle=True)
#     m3f = filter_overlap(m3)
#     np.save('/home/yelhe/script/mt/random/' + n3[m] + 'f.npy', m3f, allow_pickle=True)
#     m4 = np.load('/home/yelhe/script/mt/random/' + n4[m] + '.npy', allow_pickle=True)
#     m4f = filter_overlap(m4)
#     np.save('/home/yelhe/script/mt/random/' + n4[m] + 'f.npy', m4f, allow_pickle=True)
#     print(i)